In [5]:
import os
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
relative_parent_path = os.path.relpath(parent_directory)

In [17]:
import pandas as pd

data_path = os.path.join(relative_parent_path, 'input/Data_Entry_2017_v2020.csv')
data_entry = pd.read_csv(data_path, delimiter=',')
data_entry

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,0.171
4,00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,0.168
...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,38,M,PA,2048,2500,0.168,0.168
112116,00030802_000.png,No Finding,0,30802,28,M,PA,2048,2500,0.168,0.168
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168
112118,00030804_000.png,No Finding,0,30804,29,F,PA,2048,2500,0.168,0.168


In [18]:
import glob
import os

# looks for files with names matching
image_directory = 'input/images/images'
data_path = os.path.join(relative_parent_path, image_directory)
data_image_paths = {os.path.basename(x): x for x in glob.glob(os.path.join(data_path, '*.png'))}
data_entry['path'] = data_entry['Image Index'].map(data_image_paths.get)
data_entry


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],path
0,00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,0.143,../input/images/images/00000001_000.png
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,../input/images/images/00000001_001.png
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,../input/images/images/00000001_002.png
3,00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,0.171,../input/images/images/00000002_000.png
4,00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,0.168,../input/images/images/00000003_001.png
...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,38,M,PA,2048,2500,0.168,0.168,../input/images/images/00030801_001.png
112116,00030802_000.png,No Finding,0,30802,28,M,PA,2048,2500,0.168,0.168,../input/images/images/00030802_000.png
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168,../input/images/images/00030803_000.png
112118,00030804_000.png,No Finding,0,30804,29,F,PA,2048,2500,0.168,0.168,../input/images/images/00030804_000.png


In [19]:
import numpy as np
from itertools import chain

labels = np.unique(list(chain(*data_entry['Finding Labels'].map(lambda x: x.split('|')).tolist())))
labels = [x for x in labels if len(x) > 0]
labels

['Atelectasis',
 'Cardiomegaly',
 'Consolidation',
 'Edema',
 'Effusion',
 'Emphysema',
 'Fibrosis',
 'Hernia',
 'Infiltration',
 'Mass',
 'No Finding',
 'Nodule',
 'Pleural_Thickening',
 'Pneumonia',
 'Pneumothorax']

In [20]:
for label in labels:
    if len(label) > 1:
        data_entry[label] = data_entry['Finding Labels'].map(lambda finding: 1.0 if label in finding else 0.0)

In [21]:
df_new = pd.DataFrame(columns=data_entry.columns)

for l in labels:
    df_new = pd.concat([df_new, data_entry[data_entry[l]==1][:500]], ignore_index=True)

df_new = df_new.drop_duplicates()
df_new

/var/folders/1v/q1jh86h930b78s4n5wgsxcy80000gn/T/ipykernel_58889/811365944.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_new = pd.concat([df_new, data_entry[data_entry[l]==1][:500]], ignore_index=True)


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,...,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
0,00000011_006.png,Atelectasis,6,11,75,M,PA,2992,2991,0.143000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00000013_030.png,Atelectasis|Cardiomegaly|Emphysema|Mass|Pneumo...,8,13,56,M,AP,2500,2048,0.168000,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,00000013_035.png,Atelectasis|Pneumothorax,13,13,56,M,PA,2992,2991,0.143000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,00000019_000.png,Atelectasis|Effusion|Pleural_Thickening,0,19,66,M,PA,2992,2991,0.143000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,00000030_000.png,Atelectasis,0,30,74,M,PA,2992,2991,0.143000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7222,00003002_005.png,Effusion|Pneumothorax,19,3002,34,M,PA,2020,2021,0.194311,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7223,00003002_006.png,Effusion|Pneumothorax,20,3002,34,M,PA,2021,2020,0.194311,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7224,00003005_015.png,Pneumothorax,15,3005,49,M,AP,2500,2048,0.168000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7225,00003005_017.png,Pneumothorax,17,3005,49,M,AP,2500,2048,0.168000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [22]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(df_new, test_size=0.20, random_state=2020, stratify=df_new['Finding Labels'].map(lambda x: x[:4]))

In [23]:
train_df.loc[:, 'labels'] = train_df.apply(lambda x: x['Finding Labels'].split('|'), axis=1)
valid_df.loc[:, 'labels'] = valid_df.apply(lambda x: x['Finding Labels'].split('|'), axis=1)

In [24]:
labels_map = {i: label for i, label in enumerate(labels)}

In [25]:
image_size = 224
batch_size = 32

In [26]:
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import matplotlib.pyplot as plt
import torch.utils as utils
from torchvision.io import read_image

# Define transforms
train_transform = transforms.Compose([
   transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

valid_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
])

transform_data = transforms.Compose([transforms.ToTensor(),
                          transforms.Normalize((0.1307,), (0.3081,))])

class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.dataset_length = len(self.dataframe)
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        try:
            img_path = self.dataframe.iloc[idx]['path']
            image = Image.open(img_path).convert('RGB')
            labels = self.dataframe.iloc[idx]['labels']
            multi_hot_labels = [1 if label in labels else 0 for label in labels_map.values()]
            multi_hot_labels = torch.tensor(multi_hot_labels, dtype=torch.float32)

            if self.transform:
                image = self.transform(image)
            
            return image,  multi_hot_labels
        except Exception as e:
            print(f"Error loading data at index {idx}: {e}")
            return None, None 


# Create datasets
train_dataset = CustomDataset(train_df, transform=transform_data)
valid_dataset = CustomDataset(valid_df, transform=transform_data)


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

